<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/01_preprocessing/DF_fake_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import json
import gdown
import zipfile
import shutil
import sys
import cv2
import numpy as np
from tqdm import tqdm

In [3]:
# 1. 소스 파일이 저장된 '폴더' 경로를 지정합니다.
source_folder_path = "/content/drive/MyDrive/HECTO/src"

# 2. 시스템 경로(sys.path)에 추가합니다 (이미 추가되어 있지 않은 경우만).
if source_folder_path not in sys.path:
    sys.path.append(source_folder_path)

# 3. 이제 파일 이름(preprocessing.py)을 모듈처럼 임포트할 수 있습니다.
from preprocessing import get_aligned_face, resize_with_letterbox

print("✅ 모듈 임포트 성공!")

✅ 모듈 임포트 성공!


In [4]:
JSON_DB_PATH = "/content/drive/MyDrive/HECTO/Dataset/final_files/ff_face_metadata.json"
DF_LIST_DIR = "/content/drive/MyDrive/HECTO/Dataset/DF_lists"
DF_SAVE_BASE = "/content/drive/MyDrive/HECTO/Dataset/DF_Frames_Final"
EXTRACT_ROOT = "/content/df_temp"

# 좌표 메타데이터 로드
with open(JSON_DB_PATH, 'r') as f:
    ff_metadata = json.load(f)

# --- [2. 타겟 리스트 생성 로직] ---
def get_final_target_list():
    targets = []
    # (카테고리, 리스트파일명, ID범위, 왜곡폴더명, 라벨)
    configs = [
        ("Fake", "manipulated_videos_end_to_end_level_3_list.txt", range(0, 500), "level_3", 1),
        ("Fake", "manipulated_videos_end_to_end_level_4_list.txt", range(500, 1000), "level_4", 1),
        ("Fake", "manipulated_videos_end_to_end_level_5_list.txt", range(0, 500), "level_5", 1),
        ("Fake", "manipulated_videos_end_to_end_mix_2_distortions_list.txt", range(500, 1000), "mix_2", 1),
    ]

    for type_name, list_file, id_range, dist_folder, label in configs:
        path = os.path.join(DF_LIST_DIR, list_file)
        if not os.path.exists(path): continue
        with open(path, 'r') as f:
            for line in f:
                rel_path = line.split(' ')[0].strip()
                v_name = os.path.basename(rel_path)
                # 왜곡 키워드 매칭 확인 (파일명에 std_3, mix_2 등이 포함되어 있음)
                if dist_folder not in rel_path: continue
                try:
                    v_id_int = int(v_name.split('_')[0])
                    if v_id_int in id_range:
                        targets.append({
                            'rel_path': rel_path,
                            'v_id': str(v_id_int).zfill(3),
                            'type': type_name,
                            'dist': dist_folder,
                            'label': label
                        })
                except: continue
    return targets

In [5]:
FINAL_TARGETS = get_final_target_list()

In [6]:
# --- [3. 개별 영상 전처리 함수] ---
def process_single_df_video(local_video_path, item, num_frames=15):
    v_id = item['v_id']
    label = item['label']
    dist_folder = item['dist']
    type_name = item['type'] # Real 또는 Fake

    # 저장 경로 구조: DF_Frames_Final / {Real|Fake} / {std_3|mix_2...} / {영상아이디}
    video_folder_name = os.path.basename(local_video_path).replace('.mp4', '')
    save_folder = os.path.join(DF_SAVE_BASE, type_name, dist_folder, video_folder_name)

    # 이어하기 로직: 폴더 내 프레임 수 확인
    if os.path.exists(save_folder) and len(os.listdir(save_folder)) >= num_frames:
        return
    os.makedirs(save_folder, exist_ok=True)

    cap = cv2.VideoCapture(local_video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0: return
    interval = max(1, total // num_frames)

    coords = ff_metadata.get(v_id, [])
    frames_cache, face_crops_temp = [], []
    video_faces_found = False

    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if not ret: continue
        frames_cache.append(frame)

        face_to_save = None
        if i < len(coords) and coords[i]:
            info = coords[i]
            class FaceInfo: pass
            f_obj = FaceInfo()
            f_obj.bbox = np.array(info['bbox'])
            f_obj.kps = np.array(info['kps']) if info['kps'] else None

            face_to_save = get_aligned_face(frame, f_obj)
            if face_to_save is not None:
                video_faces_found = True
        face_crops_temp.append(face_to_save)

    if video_faces_found:
        for idx, crop in enumerate(face_crops_temp):
            if crop is not None:
                cv2.imwrite(os.path.join(save_folder, f"f{idx:03d}_{label}.jpg"), crop)
    else:
        for idx, frame in enumerate(frames_cache[:num_frames]):
            cv2.imwrite(os.path.join(save_folder, f"f{idx:03d}_{label}.jpg"), resize_with_letterbox(frame))
    cap.release()

In [7]:
def run_df_prepro_from_my_drive(part_names):
    """
    용량 문제를 해결하기 위해 루프 내부에서 하나씩 복사하고 삭제합니다.
    """
    for part_name in part_names:
        zip_local = f"/content/{part_name}.zip"

        # 1. 내 드라이브 사본 경로 (파일명 확인: '의 사본'이 붙어있는지 확인하세요)
        # 만약 '의 사본' 없이 저장하셨다면 아래 경로에서 해당 문구를 지우시면 됩니다.
        drive_copy_path = f"/content/drive/MyDrive/{part_name}.zip"

        # 이전에 남아있을지 모르는 잔여물 제거 (안전장치)
        if os.path.exists(zip_local): os.remove(zip_local)
        if os.path.exists(EXTRACT_ROOT): shutil.rmtree(EXTRACT_ROOT)

        print(f"\n🚀 [Step 1] 구드에서 로컬로 하나만 복사 중: {part_name}")

        # 파일 존재 여부 확인 후 복사
        if os.path.exists(drive_copy_path):
            shutil.copy(drive_copy_path, zip_local) # 여기서 하나만 가져옴
        else:
            print(f"❌ 파일을 찾을 수 없습니다: {drive_copy_path}")
            continue

        # 2. 압축 해제
        print(f"🔓 [Step 2] {part_name} 압축 해제 중...")
        os.makedirs(EXTRACT_ROOT, exist_ok=True)
        os.system(f'unzip -q "{zip_local}" -d "{EXTRACT_ROOT}"')

        # 3. 전처리 실행 (추출된 영상만 처리)
        for item in tqdm(FINAL_TARGETS, desc=f"Scanning {part_name}"):
            local_path = os.path.join(EXTRACT_ROOT, item['rel_path'])
            if os.path.exists(local_path):
                process_single_df_video(local_path, item)

        # 4. 즉시 삭제 (용량 확보의 핵심)
        print(f"🧹 [Step 4] {part_name} 로컬 데이터 삭제 중...")
        os.remove(zip_local)           # 로컬 ZIP 삭제
        shutil.rmtree(EXTRACT_ROOT)     # 압축 풀린 폴더 삭제
        print(f"✅ {part_name} 처리 완료 및 디스크 용량 확보 성공")

In [10]:
# --- 실행부 ---
target_parts = [
    'manipulated_videos_part_00',
    'manipulated_videos_part_01',
    'manipulated_videos_part_02',
    'manipulated_videos_part_03',
    'manipulated_videos_part_04',
    'manipulated_videos_part_05',
    'manipulated_videos_part_06',
    'manipulated_videos_part_07',
    'manipulated_videos_part_08',
    'manipulated_videos_part_09',
    'manipulated_videos_part_10',
    'manipulated_videos_part_11'
]


In [11]:
# 이제 이 함수 하나만 실행하면 됩니다.
run_df_prepro_from_my_drive(target_parts)


🚀 [Step 1] 구드에서 로컬로 하나만 복사 중: manipulated_videos_part_00
🔓 [Step 2] manipulated_videos_part_00 압축 해제 중...


Scanning manipulated_videos_part_00: 100%|██████████| 2000/2000 [00:00<00:00, 87975.17it/s]


🧹 [Step 4] manipulated_videos_part_00 로컬 데이터 삭제 중...
✅ manipulated_videos_part_00 처리 완료 및 디스크 용량 확보 성공

🚀 [Step 1] 구드에서 로컬로 하나만 복사 중: manipulated_videos_part_01
🔓 [Step 2] manipulated_videos_part_01 압축 해제 중...


Scanning manipulated_videos_part_01: 100%|██████████| 2000/2000 [15:26<00:00,  2.16it/s]


🧹 [Step 4] manipulated_videos_part_01 로컬 데이터 삭제 중...
✅ manipulated_videos_part_01 처리 완료 및 디스크 용량 확보 성공
